<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite

print("Installing folium...")
await piplite.install(['folium'])
print("✅ folium installed.")

print("Installing pandas...")
await piplite.install(['pandas'])
print("✅ pandas installed.")


Installing folium...
✅ folium installed.
Installing pandas...
✅ pandas installed.


In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


# Task 1: Mark all launch sites on a map with empty coordinates

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Loop through each launch site and add a Circle and a Marker
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    # Add circle
    folium.Circle(
        location=coord,
        radius=1000,
        color='#007849',  # SpaceX green
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # Add marker with label
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: #007849;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map
Launch sites are all in the Northern Hemisphere, but:
- Sites like CCAFS LC-40, SLC-40, and KSC LC-39A in Florida are relatively close to the Equator (around 28.5° N).
- VAFB SLC-4E in California is further north (~34.6° N).
Conclusion: All sites are in the southern U.S., closer to the Equator than most U.S. cities, which is ideal because:
- Launches near the equator take advantage of Earth’s rotational speed.
- It’s more efficient (uses less fuel) to reach geostationary orbit.

All the SpaceX launch sites (e.g., Cape Canaveral, Vandenberg AFB) are either right on the coast or a few kilometers away.

- Launching eastward over the ocean minimizes risk to people and property in case of failure.
- It provides a safe splashdown zone for boosters and capsules.
- Easier recovery of first stages (ASDS drone ship landings).

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Assign marker color based on class column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize the map centered at NASA JSC
cluster_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map)

# Add colored markers to the marker cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)
cluster_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df, add a marker to the cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color'], icon='rocket', prefix='fa'),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [14]:
import folium
from folium.plugins import MousePosition, MeasureControl

# Launch site coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# ✅ Only add marker to existing site_map
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# ✅ Add MousePosition plugin (real-time Lat/Lon in top-right)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' | Long: ',
    empty_string='Mouse over the map',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# ✅ Add MeasureControl plugin (interactive distance ruler)
site_map.add_child(MeasureControl(
    position='bottomleft',
    primary_length_unit='kilometers',
    secondary_length_unit='miles',
    active_color='red',
    completed_color='green'
))

# Show the updated map (no reset!)
site_map


In [15]:
# Cape Canaveral Beach (proxy for coast) — adjust as needed
coast_coord = (28.5625, -80.56775)


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Closest coastline coordinate (identified using MousePosition)
coastline_lat = 28.5625
coastline_lon = -80.56775

# Add a simple marker at the coastline point
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline Point',
    icon=folium.Icon(color='cadetblue', icon='globe')
).add_to(site_map)

# Display the updated map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# site_map.add_child(lines)

from folium.features import DivIcon

# Define coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
coastline_lat = 28.5625
coastline_lon = -80.56775

# Calculate distance if not done already
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Draw line from launch site to coastline
line_to_coast = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=2,
    opacity=0.6
)
site_map.add_child(line_to_coast)

# Create a distance label marker
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)

# Show map
site_map




Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


In [19]:
# Nearest railway coordinates
railway_lat = 28.56331
railway_lon = -80.58694

# Step 1: Calculate distance to railway
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Step 2: Draw a line from the launch site to the railway point
line_to_railway = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    color='green',
    weight=2,
    opacity=0.6
)
site_map.add_child(line_to_railway)

# Step 3: Create a marker at the railway point
folium.Marker(
    location=[railway_lat, railway_lon],
    popup='Nearest Railway',
    icon=folium.Icon(color='green', icon='train', prefix='fa')
).add_to(site_map)

# Step 4: Add a label showing distance
railway_distance_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:green;"><b>{:.2f} KM</b></div>'.format(distance_railway)
    )
)
site_map.add_child(railway_distance_marker)


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


In [20]:
# Nearest highway coordinates
highway_lat = 28.56329
highway_lon = -80.57078

# Step 1: Calculate distance to highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Step 2: Draw a red line from the launch site to the highway
line_to_highway = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    color='red',
    weight=2,
    opacity=0.6
)
site_map.add_child(line_to_highway)

# Step 3: Create a red marker at the highway point
folium.Marker(
    location=[highway_lat, highway_lon],
    popup='Nearest Highway',
    icon=folium.Icon(color='red', icon='road', prefix='fa')
).add_to(site_map)

# Step 4: Add a label showing distance in red
highway_distance_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:red;"><b>{:.2f} KM</b></div>'.format(distance_highway)
    )
)
site_map.add_child(highway_distance_marker)


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [21]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Launch Site: CCAFS SLC-40
launch_site_coord = [28.563197, -80.576820]

# Example: Closest city (Titusville, FL – approximated from mouse position)
city_coord = [28.6120, -80.8070]


In [22]:
city_distance = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                                    city_coord[0], city_coord[1])


In [23]:
def add_distance_annotation(map_object, from_coord, to_coord, distance_km, label='Distance', color='blue', dashed=True):
    """
    Adds a line and distance marker between two points on a Folium map.
    Does not return anything—just updates the existing map.
    """

    # Optional dashed line
    line_style = {'dash_array': '5, 5'} if dashed else {}

    # Draw the connection line
    city_line = folium.PolyLine(
        locations=[from_coord, to_coord],
        weight=2,
        color=color,
        **line_style
    )
    map_object.add_child(city_line)

    # Add a distance label marker at the destination
    city_marker = folium.Marker(
        location=to_coord,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{label}: {distance_km:.2f} KM</b></div>'
        )
    )
    map_object.add_child(city_marker)


In [24]:
launch_site_coord = (28.563197, -80.576820)
city_coord = (28.6120, -80.8070)
city_distance = calculate_distance(*launch_site_coord, *city_coord)

# Add to existing map
add_distance_annotation(
    map_object=site_map,
    from_coord=launch_site_coord,
    to_coord=city_coord,
    distance_km=city_distance,
    label='City',
    color='purple',
    dashed=True
)

# Show the updated map
site_map


In [25]:
city_line = folium.PolyLine(
    locations=[launch_site_coord, city_coord],
    weight=2,
    color='blue',
    dash_array='5, 5'  # optional dashed line
)
site_map.add_child(city_line)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


**Are launch sites in close proximity to railways?**

Yes. The nearest railway is approximately 0.99 km from the launch site, indicating that rail infrastructure is easily accessible. This proximity likely facilitates transportation of heavy rocket components and fuel, which are commonly moved via freight rail due to their bulk and safety requirements.

**Are launch sites in close proximity to highways?**

Yes. With a highway located just ~0.59 km away, the site is well connected by road. This supports the logistics chain for personnel, lighter materials, and rapid response operations, enabling efficient ground mobility in and out of the launch complex.

**Are launch sites in close proximity to the coastline?**

Yes. The launch site is just ~0.89 km from the coastline, a common strategic choice for several reasons:
- Safety: Launching over water minimizes risks to populated areas in case of failure.
- Trajectory: Coastal locations enable a wider range of launch azimuths (flight paths).
- Recovery: Easier to deploy droneships and recovery crews for booster landings.

**Do launch sites keep a certain distance away from cities?**

Yes. The nearest city is about 23.13 km away. This significant buffer helps:
- Ensure public safety in the event of launch failures or fuel explosions.
- Reduce noise and shockwave impact on civilians.
- Comply with regulatory requirements for hazardous operations near populated areas.

**Conclusion**

- The spatial distribution of launch infrastructure around CCAFS-SLC 40 reflects deliberate planning. The site is:
- Well-connected to logistical routes (rail + road),
- Strategically close to the ocean for safety and orbital flexibility,
- And safely distanced from major population centers, reflecting both engineering and regulatory best practices in aerospace facility design.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
